### Problem Descripition 

In 2012, URL shortening service Bitly partnered with the US government website USA.gov to provide a feed of anonymous data gathered from users who shorten links ending with .gov or .mil.

The text file comes in JSON format and here are some keys and their description. They are only the most important ones for this task.

|key| description |
|---|-----------|
| a|Denotes information about the web browser and operating system|
| tz | time zone |
| r | URL the user come from |
| u | URL where the user headed to |
| t | Timestamp when the user start using the website in UNIX format |
| hc | Timestamp when user exit the website in UNIX format |
| cy | City from which the request intiated |
| ll | Longitude and Latitude |

In the cell, I tried to provide some helper code for better understanding and clearer vision

-**HINT**- Those lines of code may be not helping at all with your task.

In [1]:
# I will try to retrieve one instance of the file in a list of dictionaries
import json
records = [json.loads(line) for line in open('usa.gov_click_data.json')]
# Print the first occurance
len(records)

3560

In [2]:
import pandas as pd
import numpy as np
import re

In [3]:
#     Reading all the multiple lines of the json file in the dataframe
result = pd.read_json ('usa.gov_click_data.json', lines=True)
result.head(14)

,a,c,nk,tz,gr,g,h,l,al,hh,r,u,t,hc,cy,ll,_heartbeat_,kw
0,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,US,1.0,America/New_York,MA,A6qOVH,wfLQtf,orofrog,"en-US,en;q=0.8",1.usa.gov,http://www.facebook.com/l/7AQEFzjSi/1.usa.gov/...,http://www.ncbi.nlm.nih.gov/pubmed/22415991,1.331923e+09,1.331823e+09,Danvers,"[42.576698, -70.954903]",NaN,NaN
1,GoogleMaps/RochesterNY,US,0.0,America/Denver,UT,mwszkS,mwszkS,bitly,NaN,j.mp,http://www.AwareMap.com/,http://www.monroecounty.gov/etc/911/rss.php,1.331923e+09,1.308262e+09,Provo,"[40.218102, -111.613297]",NaN,NaN
2,Mozilla/4.0 (Windows NT 6.1; MSIE 8.0; Windows...,US,1.0,America/New_York,DC,xxr3Qb,xxr3Qb,bitly,en-US,1.usa.gov,http://t.co/03elZC4Q,http://boxer.senate.gov/en/press/releases/0316...,1.331923e+09,1.331920e+09,Washington,"[38.9007, -77.043098]",NaN,NaN
3,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_8)...,BR,0.0,America/Sao_Paulo,27,zCaLwp,zUtuOu,alelex88,pt-br,1.usa.gov,direct,http://apod.nasa.gov/apod/ap120312.html,1.331923e+09,1.331923e+09,Braz,"[-23.549999, -46.616699]",NaN,NaN
4,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,US,0.0,America/New_York,MA,9b6kNl,9b6kNl,bitly,"en-US,en;q=0.8",bit.ly,http://www.shrewsbury-ma.gov/selco/,http://www.shrewsbury-ma.gov/egov/gallery/1341...,1.331923e+09,1.273672e+09,Shrewsbury,"[42.286499, -71.714699]",NaN,NaN
5,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,US,0.0,America/New_York,MA,axNK8c,axNK8c,bitly,"en-US,en;q=0.8",bit.ly,http://www.shrewsbury-ma.gov/selco/,http://www.shrewsbury-ma.gov/egov/gallery/1341...,1.331923e+09,1.273673e+09,Shrewsbury,"[42.286499, -71.714699]",NaN,NaN
6,Mozilla/5.0 (Windows NT 5.1) AppleWebKit/535.1...,PL,0.0,Europe/Warsaw,77,wcndER,zkpJBR,bnjacobs,"pl-PL,pl;q=0.8,en-US;q=0.6,en;q=0.4",1.usa.gov,http://plus.url.google.com/url?sa=z&n=13319232...,http://www.nasa.gov/mission_pages/nustar/main/...,1.331923e+09,1.331923e+09,Luban,"[51.116699, 15.2833]",NaN,NaN
7,Mozilla/5.0 (Windows NT 6.1; rv:2.0.1) Gecko/2...,None,0.0,,NaN,wcndER,zkpJBR,bnjacobs,"bg,en-us;q=0.7,en;q=0.3",1.usa.gov,http://www.facebook.com/,http://www.nasa.gov/mission_pages/nustar/main/...,1.331923e+09,1.331923e+09,NaN,NaN,NaN,NaN
8,Opera/9.80 (Ubuntu 14.04.6; Linux zbov; U; en)...,None,0.0,,NaN,wcndER,zkpJBR,bnjacobs,"en-US, en",1.usa.gov,http://www.facebook.com/l.php?u=http%3A%2F%2F1...,http://www.nasa.gov/mission_pages/nustar/main/...,1.331923e+09,1.331923e+09,NaN,NaN,NaN,NaN
9,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,None,0.0,,NaN,zCaLwp,zUtuOu,alelex88,"pt-BR,pt;q=0.8,en-US;q=0.6,en;q=0.4",1.usa.gov,http://t.co/o1Pd0WeV,http://apod.nasa.gov/apod/ap120312.html,1.331923e+09,1.331923e+09,NaN,NaN,NaN,NaN


In [4]:
#     Two lists first with the columns we need at the end, second with their corresponding columns in the dataframe
#     Note: The new cols with no direct corresponding old cols are left as empty strings in the second list
adjusted_cols_vals=['web_browser','operating_sys','from_url','to_url','city','longitude','latitude','time_zone','time_in','time_out']
adjusted_cols=['','','r','u','cy','','','tz','hc','t']

#     Renaming the cols in 'adjusted_cols' with the new cols in 'adjusted_cols_vals'
#     Note: The exception rising because of the empty strings is being handled using the <try except> method
for i in range(len(adjusted_cols)):
    try:
        result.rename(columns={adjusted_cols[i]:adjusted_cols_vals[i]},inplace=True)
    except:
        None
result.head(14)

,a,c,nk,time_zone,gr,g,h,l,al,hh,from_url,to_url,time_out,time_in,city,ll,_heartbeat_,kw
0,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,US,1.0,America/New_York,MA,A6qOVH,wfLQtf,orofrog,"en-US,en;q=0.8",1.usa.gov,http://www.facebook.com/l/7AQEFzjSi/1.usa.gov/...,http://www.ncbi.nlm.nih.gov/pubmed/22415991,1.331923e+09,1.331823e+09,Danvers,"[42.576698, -70.954903]",NaN,NaN
1,GoogleMaps/RochesterNY,US,0.0,America/Denver,UT,mwszkS,mwszkS,bitly,NaN,j.mp,http://www.AwareMap.com/,http://www.monroecounty.gov/etc/911/rss.php,1.331923e+09,1.308262e+09,Provo,"[40.218102, -111.613297]",NaN,NaN
2,Mozilla/4.0 (Windows NT 6.1; MSIE 8.0; Windows...,US,1.0,America/New_York,DC,xxr3Qb,xxr3Qb,bitly,en-US,1.usa.gov,http://t.co/03elZC4Q,http://boxer.senate.gov/en/press/releases/0316...,1.331923e+09,1.331920e+09,Washington,"[38.9007, -77.043098]",NaN,NaN
3,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_8)...,BR,0.0,America/Sao_Paulo,27,zCaLwp,zUtuOu,alelex88,pt-br,1.usa.gov,direct,http://apod.nasa.gov/apod/ap120312.html,1.331923e+09,1.331923e+09,Braz,"[-23.549999, -46.616699]",NaN,NaN
4,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,US,0.0,America/New_York,MA,9b6kNl,9b6kNl,bitly,"en-US,en;q=0.8",bit.ly,http://www.shrewsbury-ma.gov/selco/,http://www.shrewsbury-ma.gov/egov/gallery/1341...,1.331923e+09,1.273672e+09,Shrewsbury,"[42.286499, -71.714699]",NaN,NaN
5,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,US,0.0,America/New_York,MA,axNK8c,axNK8c,bitly,"en-US,en;q=0.8",bit.ly,http://www.shrewsbury-ma.gov/selco/,http://www.shrewsbury-ma.gov/egov/gallery/1341...,1.331923e+09,1.273673e+09,Shrewsbury,"[42.286499, -71.714699]",NaN,NaN
6,Mozilla/5.0 (Windows NT 5.1) AppleWebKit/535.1...,PL,0.0,Europe/Warsaw,77,wcndER,zkpJBR,bnjacobs,"pl-PL,pl;q=0.8,en-US;q=0.6,en;q=0.4",1.usa.gov,http://plus.url.google.com/url?sa=z&n=13319232...,http://www.nasa.gov/mission_pages/nustar/main/...,1.331923e+09,1.331923e+09,Luban,"[51.116699, 15.2833]",NaN,NaN
7,Mozilla/5.0 (Windows NT 6.1; rv:2.0.1) Gecko/2...,None,0.0,,NaN,wcndER,zkpJBR,bnjacobs,"bg,en-us;q=0.7,en;q=0.3",1.usa.gov,http://www.facebook.com/,http://www.nasa.gov/mission_pages/nustar/main/...,1.331923e+09,1.331923e+09,NaN,NaN,NaN,NaN
8,Opera/9.80 (Ubuntu 14.04.6; Linux zbov; U; en)...,None,0.0,,NaN,wcndER,zkpJBR,bnjacobs,"en-US, en",1.usa.gov,http://www.facebook.com/l.php?u=http%3A%2F%2F1...,http://www.nasa.gov/mission_pages/nustar/main/...,1.331923e+09,1.331923e+09,NaN,NaN,NaN,NaN
9,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,None,0.0,,NaN,zCaLwp,zUtuOu,alelex88,"pt-BR,pt;q=0.8,en-US;q=0.6,en;q=0.4",1.usa.gov,http://t.co/o1Pd0WeV,http://apod.nasa.gov/apod/ap120312.html,1.331923e+09,1.331923e+09,NaN,NaN,NaN,NaN


In [5]:
#     Replacing the np.nan in the ll (longitude,latitude) with a string of two dots to get around the error when splitting 
result['ll'] = result['ll'].replace(np.nan, '..')

#     Generating the splitted cols in a new dataframe and Adding the new dataframe to the original one
long_lat=pd.DataFrame(result.ll.tolist(), columns=['longitude', 'latitude'])
result=result.join(long_lat, lsuffix='_caller', rsuffix='_other')

#     Changing the '.' back to np.nan to keep the data consistent
result[['longitude','latitude']] = result[['longitude','latitude']].replace('.',np.nan)

result.head(14)

,a,c,nk,time_zone,gr,g,h,l,al,hh,from_url,to_url,time_out,time_in,city,ll,_heartbeat_,kw,longitude,latitude
0,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,US,1.0,America/New_York,MA,A6qOVH,wfLQtf,orofrog,"en-US,en;q=0.8",1.usa.gov,http://www.facebook.com/l/7AQEFzjSi/1.usa.gov/...,http://www.ncbi.nlm.nih.gov/pubmed/22415991,1.331923e+09,1.331823e+09,Danvers,"[42.576698, -70.954903]",NaN,NaN,42.576698,-70.954903
1,GoogleMaps/RochesterNY,US,0.0,America/Denver,UT,mwszkS,mwszkS,bitly,NaN,j.mp,http://www.AwareMap.com/,http://www.monroecounty.gov/etc/911/rss.php,1.331923e+09,1.308262e+09,Provo,"[40.218102, -111.613297]",NaN,NaN,40.218102,-111.613297
2,Mozilla/4.0 (Windows NT 6.1; MSIE 8.0; Windows...,US,1.0,America/New_York,DC,xxr3Qb,xxr3Qb,bitly,en-US,1.usa.gov,http://t.co/03elZC4Q,http://boxer.senate.gov/en/press/releases/0316...,1.331923e+09,1.331920e+09,Washington,"[38.9007, -77.043098]",NaN,NaN,38.900700,-77.043098
3,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_8)...,BR,0.0,America/Sao_Paulo,27,zCaLwp,zUtuOu,alelex88,pt-br,1.usa.gov,direct,http://apod.nasa.gov/apod/ap120312.html,1.331923e+09,1.331923e+09,Braz,"[-23.549999, -46.616699]",NaN,NaN,-23.549999,-46.616699
4,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,US,0.0,America/New_York,MA,9b6kNl,9b6kNl,bitly,"en-US,en;q=0.8",bit.ly,http://www.shrewsbury-ma.gov/selco/,http://www.shrewsbury-ma.gov/egov/gallery/1341...,1.331923e+09,1.273672e+09,Shrewsbury,"[42.286499, -71.714699]",NaN,NaN,42.286499,-71.714699
5,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,US,0.0,America/New_York,MA,axNK8c,axNK8c,bitly,"en-US,en;q=0.8",bit.ly,http://www.shrewsbury-ma.gov/selco/,http://www.shrewsbury-ma.gov/egov/gallery/1341...,1.331923e+09,1.273673e+09,Shrewsbury,"[42.286499, -71.714699]",NaN,NaN,42.286499,-71.714699
6,Mozilla/5.0 (Windows NT 5.1) AppleWebKit/535.1...,PL,0.0,Europe/Warsaw,77,wcndER,zkpJBR,bnjacobs,"pl-PL,pl;q=0.8,en-US;q=0.6,en;q=0.4",1.usa.gov,http://plus.url.google.com/url?sa=z&n=13319232...,http://www.nasa.gov/mission_pages/nustar/main/...,1.331923e+09,1.331923e+09,Luban,"[51.116699, 15.2833]",NaN,NaN,51.116699,15.283300
7,Mozilla/5.0 (Windows NT 6.1; rv:2.0.1) Gecko/2...,None,0.0,,NaN,wcndER,zkpJBR,bnjacobs,"bg,en-us;q=0.7,en;q=0.3",1.usa.gov,http://www.facebook.com/,http://www.nasa.gov/mission_pages/nustar/main/...,1.331923e+09,1.331923e+09,NaN,..,NaN,NaN,NaN,NaN
8,Opera/9.80 (Ubuntu 14.04.6; Linux zbov; U; en)...,None,0.0,,NaN,wcndER,zkpJBR,bnjacobs,"en-US, en",1.usa.gov,http://www.facebook.com/l.php?u=http%3A%2F%2F1...,http://www.nasa.gov/mission_pages/nustar/main/...,1.331923e+09,1.331923e+09,NaN,..,NaN,NaN,NaN,NaN
9,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,None,0.0,,NaN,zCaLwp,zUtuOu,alelex88,"pt-BR,pt;q=0.8,en-US;q=0.6,en;q=0.4",1.usa.gov,http://t.co/o1Pd0WeV,http://apod.nasa.gov/apod/ap120312.html,1.331923e+09,1.331923e+09,NaN,..,NaN,NaN,NaN,NaN


In [6]:
#     To be used with urls
def url_shortener(long_url):
    if long_url=='direct':
        return long_url
    start=long_url.find('//')
    end=long_url.find('/',start+2) 
#     Some URLS are already shortened or are direct so we leave them as they are
    return long_url[start+2:end]

In [7]:
#     Replacing the np.nan in the 'from_url' and 'to_url' with a string of a dot to get around the error when applying regex functions 
result[['from_url','to_url']] = result[['from_url','to_url']].replace(np.nan,'.')

#     Apply the url_shortener equation to the specified columns
result['from_url']=result['from_url'].map(url_shortener)
result['to_url']=result['to_url'].map(url_shortener)

#     Replacing the dots with np.nan again to keep the data consistent
result['from_url'].replace('.',np.nan,inplace=True)
result['to_url'].replace('.',np.nan,inplace=True)

result.head(14)

,a,c,nk,time_zone,gr,g,h,l,al,hh,from_url,to_url,time_out,time_in,city,ll,_heartbeat_,kw,longitude,latitude
0,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,US,1.0,America/New_York,MA,A6qOVH,wfLQtf,orofrog,"en-US,en;q=0.8",1.usa.gov,www.facebook.com,www.ncbi.nlm.nih.gov,1.331923e+09,1.331823e+09,Danvers,"[42.576698, -70.954903]",NaN,NaN,42.576698,-70.954903
1,GoogleMaps/RochesterNY,US,0.0,America/Denver,UT,mwszkS,mwszkS,bitly,NaN,j.mp,www.AwareMap.com,www.monroecounty.gov,1.331923e+09,1.308262e+09,Provo,"[40.218102, -111.613297]",NaN,NaN,40.218102,-111.613297
2,Mozilla/4.0 (Windows NT 6.1; MSIE 8.0; Windows...,US,1.0,America/New_York,DC,xxr3Qb,xxr3Qb,bitly,en-US,1.usa.gov,t.co,boxer.senate.gov,1.331923e+09,1.331920e+09,Washington,"[38.9007, -77.043098]",NaN,NaN,38.900700,-77.043098
3,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_8)...,BR,0.0,America/Sao_Paulo,27,zCaLwp,zUtuOu,alelex88,pt-br,1.usa.gov,direct,apod.nasa.gov,1.331923e+09,1.331923e+09,Braz,"[-23.549999, -46.616699]",NaN,NaN,-23.549999,-46.616699
4,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,US,0.0,America/New_York,MA,9b6kNl,9b6kNl,bitly,"en-US,en;q=0.8",bit.ly,www.shrewsbury-ma.gov,www.shrewsbury-ma.gov,1.331923e+09,1.273672e+09,Shrewsbury,"[42.286499, -71.714699]",NaN,NaN,42.286499,-71.714699
5,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,US,0.0,America/New_York,MA,axNK8c,axNK8c,bitly,"en-US,en;q=0.8",bit.ly,www.shrewsbury-ma.gov,www.shrewsbury-ma.gov,1.331923e+09,1.273673e+09,Shrewsbury,"[42.286499, -71.714699]",NaN,NaN,42.286499,-71.714699
6,Mozilla/5.0 (Windows NT 5.1) AppleWebKit/535.1...,PL,0.0,Europe/Warsaw,77,wcndER,zkpJBR,bnjacobs,"pl-PL,pl;q=0.8,en-US;q=0.6,en;q=0.4",1.usa.gov,plus.url.google.com,www.nasa.gov,1.331923e+09,1.331923e+09,Luban,"[51.116699, 15.2833]",NaN,NaN,51.116699,15.283300
7,Mozilla/5.0 (Windows NT 6.1; rv:2.0.1) Gecko/2...,None,0.0,,NaN,wcndER,zkpJBR,bnjacobs,"bg,en-us;q=0.7,en;q=0.3",1.usa.gov,www.facebook.com,www.nasa.gov,1.331923e+09,1.331923e+09,NaN,..,NaN,NaN,NaN,NaN
8,Opera/9.80 (Ubuntu 14.04.6; Linux zbov; U; en)...,None,0.0,,NaN,wcndER,zkpJBR,bnjacobs,"en-US, en",1.usa.gov,www.facebook.com,www.nasa.gov,1.331923e+09,1.331923e+09,NaN,..,NaN,NaN,NaN,NaN
9,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,None,0.0,,NaN,zCaLwp,zUtuOu,alelex88,"pt-BR,pt;q=0.8,en-US;q=0.6,en;q=0.4",1.usa.gov,t.co,apod.nasa.gov,1.331923e+09,1.331923e+09,NaN,..,NaN,NaN,NaN,NaN


In [8]:
#     A function to extract the OS from 'a' column
def extract_os(browser_os):
    if browser_os is np.nan or browser_os=='':
        return np.nan
#   We can find the OS inside curly brackets so we use regex here to extract any values between curly brackets
    os_list=re.findall(r'\(.*?\)',str(browser_os))
    
#   Getting rid of the curly brackets
    for os_ind in range(len(os_list)):
        os_list[os_ind]=os_list[os_ind].replace('(','').replace(')','')
    
#   joining the list of OS's in a single string and return it
#     os=''.join(os)
    if os_list:
        return os_list
    return np.nan

#     A function to extract the browsers from 'a' column
def extract_browser(browser_os):
    if browser_os is np.nan or browser_os=='':
        return np.nan
#   We can find the OS inside curly brackets so we use regex here to extract any values between curly brackets
    os_list=re.findall(r'\(.*?\)',str(browser_os))
    
#   Replacing every OS with its curly brackets with an empty string to keep only the browsers
    browsers=browser_os
    for os_loop in os_list:
        browsers=browsers.replace(os_loop,'')
    if browsers:
        return browsers
    return np.nan

In [9]:
#     Extracting the browser from 'a' column and adding this as a new column in the data
result['web_browser']=result['a'].map(extract_browser)

#     Extracting the OS from 'a' column and adding this as a new column in the data
result['operating_sys']=result['a'].map(extract_os)

#     Updating the dataset so it only includes the columns we need in the order specified
result=result[adjusted_cols_vals]

In [10]:
#     Getting rid of rows with any nulls and adjusting the index
result.dropna(
    how='any',
    inplace=True
)
result=result.reset_index().drop(columns='index')

result.head(17)

,web_browser,operating_sys,from_url,to_url,city,longitude,latitude,time_zone,time_in,time_out
0,Mozilla/5.0 AppleWebKit/535.11 Chrome/17.0.9...,"[Windows NT 6.1; WOW64, KHTML, like Gecko]",www.facebook.com,www.ncbi.nlm.nih.gov,Danvers,42.576698,-70.954903,America/New_York,1.331823e+09,1.331923e+09
1,Mozilla/4.0,[Windows NT 6.1; MSIE 8.0; Windows NT 6.1; WOW...,t.co,boxer.senate.gov,Washington,38.900700,-77.043098,America/New_York,1.331920e+09,1.331923e+09
2,Mozilla/5.0 AppleWebKit/534.52.7 Version/5.1...,"[Macintosh; Intel Mac OS X 10_6_8, KHTML, like...",direct,apod.nasa.gov,Braz,-23.549999,-46.616699,America/Sao_Paulo,1.331923e+09,1.331923e+09
3,Mozilla/5.0 AppleWebKit/535.11 Chrome/17.0.9...,"[Windows NT 6.1; WOW64, KHTML, like Gecko]",www.shrewsbury-ma.gov,www.shrewsbury-ma.gov,Shrewsbury,42.286499,-71.714699,America/New_York,1.273672e+09,1.331923e+09
4,Mozilla/5.0 AppleWebKit/535.11 Chrome/17.0.9...,"[Windows NT 6.1; WOW64, KHTML, like Gecko]",www.shrewsbury-ma.gov,www.shrewsbury-ma.gov,Shrewsbury,42.286499,-71.714699,America/New_York,1.273673e+09,1.331923e+09
5,Mozilla/5.0 AppleWebKit/535.11 Chrome/17.0.9...,"[Windows NT 5.1, KHTML, like Gecko]",plus.url.google.com,www.nasa.gov,Luban,51.116699,15.283300,Europe/Warsaw,1.331923e+09,1.331923e+09
6,Mozilla/5.0 Gecko/20100101 Firefox/10.0.2,[Windows NT 6.1; WOW64; rv:10.0.2],direct,www.nysdot.gov,Seattle,47.595100,-122.332603,America/Los_Angeles,1.319564e+09,1.331923e+09
7,Mozilla/5.0 Gecko/20120216 Firefox/3.6.27,[Macintosh; U; Intel Mac OS X 10.4; en-US; rv:...,t.co,oversight.house.gov,Washington,38.937599,-77.092796,America/New_York,1.331816e+09,1.331923e+09
8,Mozilla/5.0 Gecko/20100101 Firefox/10.0.2,[Windows NT 6.1; WOW64; rv:10.0.2],direct,www.nysdot.gov,Alexandria,38.790901,-77.094704,America/New_York,1.319564e+09,1.331923e+09
9,Mozilla/5.0 Gecko/20110420 Firefox/3.6.17,[Windows; U; Windows NT 6.1; en-US; rv:1.9.2.17],direct,toxtown.nlm.nih.gov,Marietta,33.953201,-84.517700,America/New_York,1.255770e+09,1.331923e+09


In [11]:
# Convert the timestamp to datetime format and localize all the time entries to America/New_York

result['time_in'] = pd.to_datetime(result['time_in'], unit='s')
result['time_out'] = pd.to_datetime(result['time_out'], unit='s')
result['time_in'] = pd.to_datetime(result['time_in']).dt.tz_localize('America/New_York')
result['time_out'] = pd.to_datetime(result['time_out']).dt.tz_localize('America/New_York')

result

,web_browser,operating_sys,from_url,to_url,city,longitude,latitude,time_zone,time_in,time_out
0,Mozilla/5.0 AppleWebKit/535.11 Chrome/17.0.9...,"[Windows NT 6.1; WOW64, KHTML, like Gecko]",www.facebook.com,www.ncbi.nlm.nih.gov,Danvers,42.576698,-70.954903,America/New_York,2012-03-15 14:48:38-04:00,2012-03-16 18:40:47-04:00
1,Mozilla/4.0,[Windows NT 6.1; MSIE 8.0; Windows NT 6.1; WOW...,t.co,boxer.senate.gov,Washington,38.900700,-77.043098,America/New_York,2012-03-16 17:45:41-04:00,2012-03-16 18:40:50-04:00
2,Mozilla/5.0 AppleWebKit/534.52.7 Version/5.1...,"[Macintosh; Intel Mac OS X 10_6_8, KHTML, like...",direct,apod.nasa.gov,Braz,-23.549999,-46.616699,America/Sao_Paulo,2012-03-16 18:37:48-04:00,2012-03-16 18:40:49-04:00
3,Mozilla/5.0 AppleWebKit/535.11 Chrome/17.0.9...,"[Windows NT 6.1; WOW64, KHTML, like Gecko]",www.shrewsbury-ma.gov,www.shrewsbury-ma.gov,Shrewsbury,42.286499,-71.714699,America/New_York,2010-05-12 13:53:31-04:00,2012-03-16 18:40:51-04:00
4,Mozilla/5.0 AppleWebKit/535.11 Chrome/17.0.9...,"[Windows NT 6.1; WOW64, KHTML, like Gecko]",www.shrewsbury-ma.gov,www.shrewsbury-ma.gov,Shrewsbury,42.286499,-71.714699,America/New_York,2010-05-12 13:55:06-04:00,2012-03-16 18:40:52-04:00
...,...,...,...,...,...,...,...,...,...,...
2741,Mozilla/4.0,[Windows NT 6.1; MSIE 7.0; Windows NT 6.0; SLC...,www.shrewsbury-ma.gov,www.shrewsbury-ma.gov,Shrewsbury,42.286499,-71.714699,America/New_York,2010-05-12 13:53:31-04:00,2012-03-16 19:40:40-04:00
2742,Mozilla/4.0,[Windows NT 6.1; MSIE 7.0; Windows NT 6.0; SLC...,www.shrewsbury-ma.gov,www.shrewsbury-ma.gov,Shrewsbury,42.286499,-71.714699,America/New_York,2010-05-12 13:55:06-04:00,2012-03-16 19:40:40-04:00
2743,Mozilla/4.0,[Windows NT 6.1; MSIE 9.0; Windows NT 6.1],direct,www.fda.gov,Paramus,40.944500,-74.070000,America/New_York,2011-03-28 07:47:59-04:00,2012-03-16 19:40:41-04:00
2744,Mozilla/5.0 AppleWebKit/535.11 Chrome/17.0.9...,"[Windows NT 5.1, KHTML, like Gecko]",www.facebook.com,www.okc.gov,Oklahoma City,35.471500,-97.518997,America/Chicago,2011-06-08 10:50:47-04:00,2012-03-16 19:40:44-04:00


In [12]:
result.count()

web_browser      2746
operating_sys    2746
from_url         2746
to_url           2746
city             2746
longitude        2746
latitude         2746
time_zone        2746
time_in          2746
time_out         2746
dtype: int64

In [13]:
#     Creating a csv file for the dataframe
result.to_csv('usa.gov_click_data.csv',index=False)

## Required

Write a script can transform the JSON files to a DataFrame and commit each file to a sparete CSV file in the target directory and consider the following:

        

All CSV files must have the following columns
- web_browser 
        The web browser that has requested the service
- operating_sys
        operating system that intiated this request
- from_url  ----

        The main URL the user came from

    **note**:

    If the retrived URL was in a long format `http://www.facebook.com/l/7AQEFzjSi/1.usa.gov/wfLQtf`

     make it appear in the file in a short format like this `www.facebook.com`
     
    
- to_url  ----

       The same applied like `to_url`
   
- city  ----

        The city from which the the request was sent
    
- longitude  ----

        The longitude where the request was sent
- latitude  ----

        The latitude where the request was sent

- time_zone  ----
        
        The time zone that the city follow
        
- time_in ----

        Time when the request started
- time_out ----
        
        Time when the request is ended
        
        
**NOTE** :

Because that some instances of the file are incomplete, you may encouter some NaN values in your transforamtion. Make sure that the final dataframes have no NaNs at all.